In [ ]:
import numpy as np
import pandas as pd
from numpy.random import default_rng
rng = default_rng()
import math
from math import log as log
from numpy.linalg import det as det
from IPython.display import Markdown, display
from sympy import symbols, solve
def printmd(string):
    display(Markdown(string))

### !Design matrix

In [ ]:
cluster_1 = np.random.normal(-50, 0.04, (5, 20))
cluster_2 = np.random.normal(300, 0.7, (10, 20))
discriminating_data = np.concatenate((cluster_1, cluster_2))
non_discriminating_data = np.random.normal(0, 1, (15, 80))
X = np.concatenate( (discriminating_data, non_discriminating_data), axis=1)
(n,p) = X.shape

### HYPERPARAMETERS (without alpha, determined later)


In [ ]:
#Prior of the model on the discriminatory variables : a gaussian vector
mu_0 = np.array([np.median(X[:,j]) for j in range(p)]).reshape(p, 1)  #mean for the gaussian vector
h_1 = 1000   #multiplicatory coefficient for its variance-covariance Sigma
delta = 3   #mean of the Inverse Wishart prior for Sigma
kappa_1 = 0.0007   #variance multiplicator for the variance covariance matrix of the Inverse Wishart
Q_1 = kappa_1 * np.identity(p) #variance covariance matrix of the inverse Wishart
t = 5 #number of intermediate steps for launch state of latent vector c allocation update
gamma_total_iter = 20 # number of Metropolis updates of gamma vector of variable selection (authors say above 20 minimal improvement)
c_total_iter = 5 # number of updates of sample allocation vector c (authors of paper say above 5 minimal improvement)

#Prior of the model on non-discriminatory variables : a gaussian vector
h_0 = 100  #multiplicatory coefficient for its variance-covariance Omega
a=3        #first parameter of the Inverse Gamma prior on the constant variance sigma² of the non discriminatory (and assumed independent) elements
b = 0.2 #second parameter of the Inverse Gamma prior 

#Prior of gamma
omega = 10/p

### Functions 

In [ ]:
def prior_gamma(gamma):
    prior = 1
    for j in range(p):
        gamma_j = gamma[j]
        prior *= omega**gamma_j*(1-omega)**gamma_j
    return prior


def log_likelihood(X, gamma, c):
    L = (-n*p/2)*math.log(math.pi)
    K = len(np.unique(c))
    p_gamma = int(np.sum(gamma))
    gamma_indices = np.argwhere(gamma).transpose()[0]
    gammaC_indices = np.argwhere(gamma==0).transpose()[0]
    mu_0gamma = mu_0[gamma_indices]
    mu_0gammaC = mu_0[gammaC_indices]
    Q_1gamma = Q_1[gamma_indices, :][:, gamma_indices] 


    for k in range(1, K+1):
        C_k = np.argwhere(c==k)
        n_k = len(C_k)
        x_kgamma = X[k-1, gamma_indices]

        H_kgamma = (h_1 * n_k + 1)**(-p_gamma/2)
        for j in range(1, p_gamma + 1):
            H_kgamma *= math.gamma( (n_k + delta + p_gamma -j)/2) / math.gamma( (delta + p_gamma -j)/2 )
        
        log_H_0gammaC = ( -(p - p_gamma)/2 )*log(h_0*n + 1) + ( a*(p-p_gamma) )*log(b)
        for j in range(1, p - p_gamma + 1):
            log_H_0gammaC += log(math.gamma(a+n/2)) - log(math.gamma(a))
            
        S_kgamma = n_k/(h_1*n_k +1)*(mu_0gamma - np.mean(x_kgamma))*np.transpose((mu_0gamma - np.mean(x_kgamma)))
        for i in C_k:
            x_igamma = X[i, gamma_indices]
            S_kgamma += (x_igamma - np.mean(x_kgamma))*np.transpose(x_igamma - np.mean(x_kgamma))
            
        log_S_0gammaC = 0
        for j in range(1, p - p_gamma + 1):
            sum_x = 0
            j_gammaC = gammaC_indices[j-1] #jth non discriminatory variable
            mu_0jgammaC = mu_0gammaC[j-1]
            for i in range(1, n+1):
                x_ijgammaC = X[i-1, j_gammaC]
                x_jgammaC = np.mean(X[:, j_gammaC])
                sum_x += (x_ijgammaC - np.mean(x_jgammaC))**2
            log_S_0gammaC += log(b + 1/2*(sum_x + n/(h_0*n+1)*(mu_0jgammaC - np.mean(x_jgammaC))**2))

        L += log(H_kgamma) + ((delta + p_gamma-1)/2)*log(abs(det(Q_1gamma))) - ((n_k + delta + p_gamma -1)/2)*log(abs(det(Q_1gamma + S_kgamma)))
        
    L += log_H_0gammaC - (a+n/2)*log_S_0gammaC
    return L


def log_conditional_aposteriori_gamma(X, gamma, c):
    return log_likelihood(X, gamma, c) + log(prior_gamma(gamma))


def gamma_single_iter(gamma):
    """ Stochastic update Metropolis"""
    gamma_size = len(gamma)

    #stochastic update
    random = np.random.random()
    gamma_new = gamma.copy()
    if random < 1/2 and len(np.argwhere(gamma)) > 0 and len(np.argwhere(gamma)) < len(gamma):
        #we swap a 0 and a 1
        gamma_zeros = np.argwhere(gamma==0)
        gamma_ones = np.argwhere(gamma)
        pick_zero = rng.choice(gamma_zeros)
        pick_one = rng.choice(gamma_ones)
        gamma_new[pick_zero] = 1 
        gamma_new[pick_one] = 0
        
    else: 
        
        pick = rng.choice(gamma_size)
        gamma_new[pick] = abs(gamma_new[pick] - 1)
    
    #apply metropolis probability of acceptance of the new array
    random = np.random.random()
    new_log_likelihood = log_conditional_aposteriori_gamma(X, gamma_new, c)
    print('-> (gamma) new L = ', new_log_likelihood)
    former_log_likelihood = log_conditional_aposteriori_gamma(X, gamma, c)
    print('-> (gamma) old L = ', former_log_likelihood)
    if new_log_likelihood - former_log_likelihood > 0:
        decision_threshold = 1
    else:
        decision_threshold = math.exp(new_log_likelihood - former_log_likelihood)
    if random <= decision_threshold:
        print('-> gamma_new is retained')
        gamma = gamma_new
    else:
        print('-> gamma is unchanged')
    return gamma


def c_update(n, c):
    
    samples = [h for h in range(n)]
    i, l = rng.choice(samples, 2, replace = False)

    # needed values for computation of a_csplit_c and a_cmerge_c
    p_gamma = int(np.sum(gamma))
    gamma_indices = np.argwhere(gamma).transpose()[0]
    mu_0gamma = mu_0[gamma_indices]
    Q_1gamma = Q_1[gamma_indices, :][:, gamma_indices] 
    x_igamma = X[i, gamma_indices]
    x_lgamma = X[l, gamma_indices]
    S_igamma = (1/(1+h_1))*(x_igamma - mu_0gamma)*np.transpose(x_igamma - mu_0gamma)
    S_lgamma = (1/(1+h_1))*(x_lgamma - mu_0gamma)*np.transpose(x_lgamma - mu_0gamma)
    S_ilgamma = (1/(1+2*h_1))*((x_igamma - mu_0gamma)*np.transpose(x_igamma - mu_0gamma) + 
                               (x_lgamma - mu_0gamma)*np.transpose(x_lgamma - mu_0gamma) + 
                               h_1*(x_igamma - x_lgamma)*np.transpose(x_igamma - x_lgamma))
    vec = [(math.gamma((delta + p_gamma + 1 - j)/2)**2/(math.gamma((delta + p_gamma -j)/2)*math.gamma((delta + p_gamma +2 -j)/2))) for j in range(1, p_gamma +1)]
    prod_vec = np.prod(vec)

    if c[i] == c[l]:

        c_split = c.copy()
        c_split[i] = max(c_split) + 1  
        split_measure = math.exp(log(alpha) + log(prod_vec) + p_gamma*log((1+2*h_1)**(1/2)/(1+h_1)) +
                                      ((delta + p_gamma -1)/2)*log(abs(det(Q_1gamma))) + 
                                      ((delta + p_gamma +1)/2)*log(abs(det(Q_1gamma + S_ilgamma))) - 
                                      ((delta + p_gamma)/2)*log(abs(det(Q_1gamma + S_igamma)*det(Q_1gamma + S_lgamma))))
        a_csplit_c = min([1, split_measure])

        acceptance_thres = rng.random()
        if a_csplit_c >= acceptance_thres:
            c = c_split
            print('-> split was accepted, c is now ', c)
        
        return c, split_measure

    else:
        c_merge = c.copy()
        c_merge[i] = c_merge[l]
        a_cmerge_c = min([1, math.exp((-1)*(log(alpha) + log(prod_vec) + p_gamma*log((1+2*h_1)**(1/2)/(1+h_1)) +
                                      ((delta + p_gamma -1)/2)*log(abs(det(Q_1gamma))) + 
                                      ((delta + p_gamma +1)/2)*log(abs(det(Q_1gamma + S_ilgamma))) - 
                                      ((delta + p_gamma)/2)*log(abs(det(Q_1gamma + S_igamma)*det(Q_1gamma + S_lgamma)))))])

        acceptance_thres = rng.random()
        if a_cmerge_c >= acceptance_thres:
            c = c_merge
            print('-> merge was accepted, c is now ', c)
            
        return c, -1
    
    
def reindexing(c):
    reindexing = {}
    new_label = 1
    for label in np.unique(c):
        reindexing[label] = new_label
        new_label += 1
    for i in range(len(c)):
        c[i] = reindexing[c[i]]
    return c


def chain_alter(gamma, n, c, loop_num):
    
    c_storage = []
    gamma_storage = []
    split_measures = []
    
    for m in range(loop_num):
        print(f"**RUNNING {m}th iteration")
        for iter in range(gamma_total_iter):
            print(f"{iter} iteration of gamma")
            gamma = gamma_single_iter(gamma)
            gamma_storage.append(gamma)
            printmd("**Num. of significant variables**")
            print(sum(gamma == 1))
            if sum(gamma ==1) == len(gamma):
                return "chain diverged too much so stopping to avoid kernel breakdown"
        
        for iter in range(c_total_iter): 
            print(f"{iter} iteration of c")
            updte = c_update(n, c)
            c = reindexing(updte[0])
            c_storage.append(c)
            split_measures.append(updte[1])
            printmd("**Clusters alloc**")
    
    return c, gamma, c_storage, gamma_storage, split_measures

### Chain hyperparameters

In [ ]:
gamma_total_iter = 20 # number of Metropolis updates of gamma vector
c_total_iter = 200 # number of stochastic explorations in c at each iteration

### Finding adequate value for parameter alpha

In [ ]:
c = np.arange(1, n+1)
c[[i for i in range(int(len(c)/2))]] = 1
gamma = np.zeros(p)
gamma[rng.choice(np.arange(0, p), 10, replace = False)] = 1

#alpha par défaut et nombre d'itérations initiales :
alpha = 1
chain_iter = 3

_, _, _, _, split_measures = chain_alter(gamma, n, c, chain_iter)

split_measure = np.quantile(np.array(split_measures)[[split_measures[i] != -1 for i in range(len(split_measures))]], 0.5)
x = symbols('x')
expr = x*split_measure - 1/4
sol = solve(expr)

alpha = float(sol[0])

In [ ]:
print(alpha)

### Running the chain with adequate configuration

In [ ]:
c = np.arange(1, n+1)
c[[i for i in range(int(len(c)/2))]] = 1
gamma = np.zeros(p)
gamma[rng.choice(np.arange(0, p), 10, replace = False)] = 1

# nombre d'itérations de la chaîne :
chain_iter = 50

c, gamma, c_storage, gamma_storage, _ = chain_alter(gamma, n, c, chain_iter)

### !Saving and plotting the results

In [ ]:
#np.savetxt("gamma_storage_" + "config5" + ".csv", gamma_storage, delimiter=",")
#np.savetxt("c_storage_" + "config5" + ".csv", c_storage, delimiter=",")

import matplotlib.pyplot as plt

num_gamma = []
for i in range(len(gamma_storage)):
    num_gamma.append(sum(gamma_storage[i] == 1))
    
num_c_distinct = []
for i in range(len(c_storage)):
    num_c_distinct.append(len(np.unique(c_storage[i])))

In [ ]:
plt.plot([i/gamma_total_iter for i in range(len(num_gamma))], num_gamma)
plt.title("Evolution of number of selected variables (in Gamma) when alpha = " + str(alpha))
plt.xlabel("Iteration")
plt.ylabel("Number of selected variables")
plt.show()

In [ ]:
plt.plot([i/c_total_iter for i in range(len(num_c_distinct))], num_c_distinct)
plt.title("Evolution of number of clusters (in c) when alpha = " + str(alpha))
plt.xlabel("Iteration")
plt.ylabel("Number of clusters")
plt.show()